# Generate `prof` files from the analysis-ready BGC-Argo dataset

In [3]:
dir_in = '~/analysis-ready-bgc-argo-dataset/'
file_in = 'AR6901474.nc'
vars_in = ['TEMP_AR','PSAL_AR']
vars_out = ['temp','salt']
dep_out = 250

In [4]:
import pandas as pd
import xarray as xr
import numpy as np



# read data and limit to the specified depth
ds = xr.open_dataset(dir_in+file_in).sel(depth=slice(None,dep_out))

for i in range(len(vars_in)): # loop over variables
    lines = [] # initialize for each variable
    for j in range(ds.sizes['time']): # loop over time
        valid = ~np.isnan(ds[vars_in[i]][j,:].values) # array of boolean for valid samples
        if np.sum(valid) > 0: # if a profile contains real value(s)
            # add date/time for every profile entry
            lines.append(str(pd.to_datetime(ds['time'][j].values).strftime('%Y-%m-%d %H:%M:%S')) + ' ' + str(np.sum(valid)) + ' 2')
            for k in range(ds.sizes['depth']): # loop over depths
                if ~np.isnan(ds[vars_in[i]][j,k].values): # if the value is real (skip if NaN)
                    lines.append('-' + str(ds['depth'][k].values) + ' ' + str(ds[vars_in[i]][j,k].values)) # depth in negative and the data value
    with open(vars_out[i]+'_prof_'+file_in[:-3]+'.dat', 'w') as file:
        for line in lines:
            file.write(line + '\n')